In [1]:
from hello.hello3 import HelloApp
from hello import hello3
from hello import hello
from hello.pid.ag_pid import agpid
from hello.pid.ag_pid.agpid import *

In [2]:
def runagpid(r):
    try:
        r.doall()
    except Exception as e:
        import traceback
        traceback.print_exc()
    finally:
        return r

    
def run_80Ltests(ip='71.189.82.196:85'):
    p = 60
    i = 19
    d = 0
    app = hello3.open_hello(ip)
    t1 = TempAutoToAutoTest(p, i, d, 39, PIDTest.CURRENT_PV, app, name="80L Temp Verif")
    t2 = TempAutoToAutoTest(p, i, d, 37, 39, app, name="80L Temp Verif")   
    r = PIDRunner([t1, t2], wb_name='temp_80L_test.xlsx')
    return runagpid(r)
    

In [3]:
#r = run_80Ltests()
_minute = 60
_hour = 60* _minute

In [4]:
def a2a_fmt(a2a):
    # reference for A2A configuration Settings
    a2a.pretest_stabilize           = 30 * _minute
    a2a.pretest_stabilize_timeout   = 60 * _minute
    a2a.pretest_stabilize_margin    = 0.2
    a2a.settle_time                 = 30 * _minute
    a2a.timeout                     = 3 * _hour
    a2a.error_band                  = 0.1
    a2a.mintime                     = 2 * _hour
    a2a.agitation_sp                = 12


def run_80Ltests2(ip='71.189.82.196:85'):
    TempOffToAutoTest.SHORT_NAME = "O2A"  # hotfix
    p = 60
    i = 35
    d = 0
    app = hello3.open_hello(ip)
    app.retry_count = -1
    
    t0 = TempOffToAutoTest(p, i, d, 26, 37, app)
    t0.pretest_stabilize_timeout   = 18 * _hour
    t0.pretest_stabilize_margin    = 0.2
    t0.settle_time                 = 60 * _minute
    t0.timeout                     = 3 * _hour
    t0.error_band                  = 0.1
    t0.mintime                     = 3 * _hour
    t0.wait_for_cooling            = True
    t0.agitation_sp                = 12
    
    # t0 sets up t1, t1 sets up t2
    # so pretest stable time can be short
    # previous testing showed that mintime
    # was a bit short, so bump to 2 hr and 
    # shrink error band to 0.1
    t1 = TempAutoToAutoTest(p, i, d, 37, 35, app)
    t1.pretest_stabilize = 10*_minute
    t1.mintime = _hour * 2
    t1.error_band = 0.1
    
    t2 = TempAutoToAutoTest(p, i, d, 35, 37, app)   
    t2.pretest_stabilize = 10 * _minute
    t2.mintime = 2 * _hour
    t2.error_band = 0.1
    
    r = PIDRunner([t0, t1, t2], wb_name='temp_80L_test_80L_a2a_60.35.0.xlsx')
    return runagpid(r)


In [5]:
def run_80Ltests3(r):
    ip='71.189.82.196:85'
    p = 60
    i = 35
    d = 0
    app = hello3.open_hello(ip)
    app.retry_count = -1
    r2 = PIDRunner(wb_name='temp_80L_test_80L_a2a_60.35.0.xlsx')
    t1 = TempAutoToAutoTest(p, i, d, 37, 35, app)
    t1.pretest_stabilize = 2*_hour
    t1.mintime = _hour * 2
    t1.error_band = 0.1
    
    t2 = TempAutoToAutoTest(p, i, d, 35, 37, app)   
    t2.pretest_stabilize = 10 * _minute
    t2.mintime = 2 * _hour
    t2.error_band = 0.1
    
    r2.add_test(t1)
    r2.add_test(t2)
    try:
        r2.runall()
    except:
        raise
    r2._tests.insert(0, r._tests[0])
    r2.plotall()
    r2.chartbypid()

In [6]:
r=run_80Ltests2()

01/13/17 16:41:33 INFO <runall>: Running test 1 of 3: P:60.000 I:35.000 D:0.0000 SP:37
01/13/17 16:41:33 INFO <run>: Beginning test run for OffToAuto Start:26.0 SP:37.0
01/13/17 16:41:33 INFO <setup_settings>: Setting PID Settings
01/13/17 16:41:36 INFO <off>: Setting Temperature to Off
01/14/17 05:32:06 INFO <start_test>: Starting Test
01/14/17 05:32:06 INFO <auto>: Setting Temperature to 37 C
01/14/17 05:32:06 INFO <wait_for_settle>: Waiting for PV to settle
01/14/17 08:47:09 INFO <run>: Finished Test
01/14/17 08:47:09 INFO <runall>: Successful test
01/14/17 08:47:09 INFO <runall>: Running test 2 of 3: P:60.000 I:35.000 D:0.0000 SP:35
01/14/17 08:47:09 INFO <run>: Beginning test run for AutoToAuto Start:37.0 SP:35.0
01/14/17 08:47:09 INFO <setup_settings>: Setting PID Settings
01/14/17 08:47:12 INFO <setup_test>: Setting Temperature to AUTO 37 C and waiting until stable for 600 seconds (3600 second timeout)
01/14/17 08:47:12 INFO <auto>: Setting Temperature to 37 C
01/14/17 08:47:12 